In [1]:
import os 

NUM_CPU = len(os.sched_getaffinity(0))
print(f'Number of CPUs: {NUM_CPU}')
NUM_THREADS = 1 
os.environ["OMP_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["MKL_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["OPENBIAS_NUM_THREADS"] = str(NUM_THREADS)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(NUM_THREADS)
os.environ["NUMEXPR_NUM_THREADS"]  = str(NUM_THREADS)

NUM_PROCESS = int(NUM_CPU // NUM_THREADS) 

Number of CPUs: 12


In [2]:
import numpy as np
import math
import time
from copy import deepcopy
np.set_printoptions(suppress=True)
from multiprocessing import Pool

def beta_true_gen(N,p,K,gamma):
    '''
    Generate beta_true
    '''
    np.random.seed(0)  
    beta_true = np.zeros([p+1,K-1])
    for k in range(K-1):
        beta_true[:,k] = np.random.normal(size = [p+1,]) 
        beta_true[:,k] = beta_true[:,k]/np.linalg.norm(beta_true[:,k])
    beta_true[0,:] = beta_true[0,:] + gamma*np.log(N)
    return beta_true

def X_gen(N, p=5, rho=0.5):
    '''
    Generate features X
    '''
    mean = np.zeros(p)
    cov = np.zeros([p,p])
    for i in range(p):
        for j in range(i,p):
            cov[i,j]=rho**(np.abs(i-j))
    cov = cov+cov.T-np.eye(p)        
    X = np.random.multivariate_normal(mean, cov, (N,)) 
    return(X)

def get_onehot(y,baseclass = None):
    '''One-hot'''
    idx = np.squeeze(y)
    ss = len(y)
    nclass = len(np.unique(y))
    z = np.zeros([ss,nclass])
    z[np.arange(ss),idx] = 1  
    ls_class = list(np.arange(nclass))
    if baseclass is None:
        baseclass = K-1
    _ = ls_class.pop(baseclass)
    return z[:,ls_class]

def mle_multilogistic_opt_ic(x,y,K0_pval,baseclass = None): 
    '''GMLE (NR algorithm)'''
    ss,ncov = x.shape  
    K = len(np.unique(y))    
    y = get_onehot(y,baseclass) 
    dist=1.0; niter=0
    beta0 = np.zeros([ncov*(K-1),1])  
    alpha0 = np.log((1/K0_pval-1)/(K-1)) 
    beta0[np.arange(K-1)*ncov] = alpha0 
    while (dist>1.0e-6) & (niter<50):
        niter=niter+1
        beta0mat = (beta0.reshape([(K-1),ncov]).T)*1.0
        link_mu = x@beta0mat  
        prob = np.exp(link_mu);prob=prob/(1+np.sum(prob,axis = 1,keepdims=True))
        resid = y-prob
        D1=((x.T@resid/ss).T.flatten()).reshape([-1,1]) # The first-order derivative
        Xrep = x.reshape([1,ss,ncov]) * np.ones([K-1,1,1]) 
        XMAT = (prob.T).reshape([K-1,ss,1]) * Xrep  
        XMAT = (XMAT.transpose([1,0,2])).reshape([ss,-1])  
        D2 = -(XMAT.T @ XMAT/ss)  # The second-order derivative
        
        for i in range(K-1):    
            probtmp = (prob[:,i])*1.0
            weight = np.sqrt(probtmp*(1-probtmp))
            wx = weight.reshape([ss,1]) ; wx = wx * x
            D2[i*ncov:(i+1)*ncov,i*ncov:(i+1)*ncov] = wx.T@wx/ss   

        step = (np.linalg.inv(D2+1.0e-6*np.eye(ncov*(K-1))))@D1   
        beta1 = beta0+step
        assert beta1.shape==(ncov*(K-1),1),'shape is wrong'
        dist = np.mean(np.abs(beta1-beta0))
        beta0 = beta1
    return beta0.reshape([(K-1),ncov]).T, dist, niter  

def gd_multilogistic_opt_ic(x,y,K0_pval,baseclass, alpha): 
    '''GMLE (GD algorithm)'''
    ss,ncov = x.shape  
    K = len(np.unique(y)) 
    y = get_onehot(y,baseclass) 
    dist = 1.0; niter = 0
    beta0 = np.zeros([ncov*(K-1),1]) 
    alpha0 = np.log((1/K0_pval-1)/(K-1))
    beta0[np.arange(K-1)*ncov] = alpha0
    while (dist>1.0e-6) & (niter<1000):
        niter=niter+1
        beta0mat = (beta0.reshape([(K-1),ncov]).T)*1.0
        link_mu = x@beta0mat  
        prob = np.exp(link_mu);prob=prob/(1+np.sum(prob,axis = 1,keepdims=True))
        resid = y-prob
        D1=((x.T@resid/ss).T).reshape([-1,1])
        beta1 = beta0 + alpha * D1
        assert beta1.shape==(ncov*(K-1),1),'shape is wrong'
        dist = np.mean(np.abs(beta1-beta0))
        beta0 = beta1
    return beta0.reshape([(K-1),ncov]).T, dist, niter 


def mle_logistic_cpu_ic(k):
    '''PMLE'''
    np.random.seed(k)
    # Subsample
    idx_k = np.where(Y==k)[0] 
    idx_k = np.concatenate((idx_0,idx_k)) 
    x = X[idx_k]; y = Y[idx_k]
    # Optimization
    ss,ncov = x.shape  
    y = y.reshape(ss,1)
    y = 1*(y!=0) 
    dist=1.0
    niter=0
    beta0 = np.zeros([ncov,1])
    alpha0 = np.log(1/K0_pval-1)
    beta0[0] = alpha0
    
    while (dist>1.0e-6) & (niter<50):
        niter=niter+1
        link_mu = x@beta0  
        prob = np.exp(link_mu);prob=prob/(1+prob)
        resid=y-prob
        D1=x.T@resid/ss  # The first-order derivative
        weight = np.sqrt(prob*(1-prob))
        wx  = weight*x 
        del weight
        D2=wx.T@wx/ss+1.0e-6*np.eye(ncov)  # The second-order derivative
        del wx 
        step=np.linalg.inv(D2)@D1
        beta1=beta0+step
        assert beta1.shape==(ncov,1),'shape is wrong'
        dist=np.mean(np.abs(beta1-beta0))
        beta0=beta1
    return beta1.reshape([ncov,])

In [3]:
'''
N = 10**5, 2*10**5, 5*10**5
'''
N = 10**5    # Sample size
p = 10       # Feature dimension
K = 11       # Number of Classes
gamma = -0.5 # Rareness
nsimu = 100
alpha = 40
baseclass = 0
K0_pval = 0.9
a = 25

t_nr = np.zeros(nsimu)
t_gd = np.zeros(nsimu)
t_pmle = np.zeros(nsimu)

beta_hat = np.zeros([nsimu,p+1,(K-1)])
beta_hat_nr = np.zeros([nsimu,p+1,(K-1)])
beta_hat_gd = np.zeros([nsimu,p+1,(K-1)])
dist = np.zeros([nsimu]); dist_nr = 0*dist; dist_gd = 0*dist
niter = np.zeros([nsimu]);  niter_nr = 0*niter;  niter_gd = 0*niter
num_pos_idxs = np.zeros([nsimu,K])

beta_true = beta_true_gen(N,p,K,-0.5) 

t3 = time.time()
for b in range(nsimu):
    if (b % a==0): t1 = time.time()
    
    np.random.seed(b)
    X = X_gen(N,p)
    X = np.hstack([np.ones([N,1]),X]) 

    prob = np.exp(X@beta_true) 
    prob = np.hstack([np.ones([N,1]),prob])  
    prob = prob/np.sum(prob,1).reshape([N,1])
    prob = np.cumsum(prob,1)   
    Y = (np.random.uniform(size = [N,1])<prob).astype(np.int16)
    Y = np.argmax(Y,1) 
    idx_0 = np.where(Y==0)[0] 
    
    cls = np.zeros([N,K])
    cls[np.arange(N),Y] = 1 
    num_pos_idxs[b] =  np.sum(cls,0) 
    del cls
    
    ## GMLE: NR
    t5 = time.time()
    beta_hat_nr[b], dist_nr[b], niter_nr[b] = mle_multilogistic_opt_ic(X,Y,K0_pval,baseclass = baseclass)
    t6 = time.time(); t_nr[b] = t6-t5
    
    ## GMLE: GD
    t5 = time.time()
    beta_hat_gd[b], dist_gd[b], niter_gd[b] = gd_multilogistic_opt_ic(X,Y,K0_pval,baseclass,alpha)
    t6 = time.time(); t_gd[b] = t6-t5
    
    ## PMLE: NR
    np.random.seed(b)
    t5 = time.time()
    with Pool(K-1) as pool:
        beta_hat[b] = np.array(pool.map(mle_logistic_cpu_ic, [k for k in range(1,K)])).T

    t6 = time.time(); t_pmle[b] = t6-t5
    
    if (b % a==0): 
        t2 = time.time(); print('epoch {}: {}s'.format(b,np.round(t2-t1, 2)))

t4 = time.time(); print(f'\n{np.round(t4-t3,4)}s')

epoch 0: 10.65s

10.6564s


In [ ]:
# np.savez(f'./N{int(N/10**5)}_K{K}_p{p}_K0_pval[{K0_pval}].npz', 
#          beta_pmle=beta_hat, beta_nr=beta_hat_nr, beta_gd=beta_hat_gd, beta_true=beta_true,
#          t_pmle = t_pmle, t_nr = t_nr,t_gd = t_gd, num_pos_idxs=num_pos_idxs,
#          niter_pmle = niter, niter_nr = niter_nr,niter_gd = niter_gd,
#         dist_pmle = dist, dist_nr = dist_nr,dist_gd = dist_gd)